此部分用于梳理实验的逻辑，我们的实验更新了关于时序数据的部分，增大了实验的数据feature

第一步是分割图像并计算每个region的tree_height,building_height

In [ ]:
pip install rasterio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# 检查文件是否存在
file_path = '/content/drive/MyDrive/UTCI_prediction/data/spatial_images/landcover9.tif'
if os.path.exists(file_path):
    print("文件存在")
else:
    print("文件不存在，检查路径或文件名")

文件存在


In [ ]:
import pandas as pd
import numpy as np
import rasterio
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu

def read_tif(file_path):
    """读取TIF文件并返回数据数组。"""
    with rasterio.open(file_path) as src:
        return src.read(1)  # 读取第一层数据

def segment_by_landcover(image):
    """Segment the landcover image based on its unique categorical values."""
    labels = label(image)  # Automatically label each area
    return labels

# 读取空间数据
landcover = read_tif('/content/drive/MyDrive/UTCI_prediction/data/spatial_images/landcover9.tif')
tree_height = read_tif('/content/drive/MyDrive/UTCI_prediction/data/spatial_images/tree height_9.tif')
building_height = read_tif('/content/drive/MyDrive/UTCI_prediction/data/spatial_images/building height_9.tif')
dem = read_tif('/content/drive/MyDrive/UTCI_prediction/data/spatial_images/DEM_9.tif')

segmented_landcover = segment_by_landcover(landcover)

# 读取CSV文件
csv_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/Yifan_updated_data_with_timestamps.csv')

features_list = []
regions = regionprops(segmented_landcover, intensity_image=tree_height)

for index, row in csv_data.iterrows():
    for region in regions:
        features = {
            'region_id': region.label,
            'tree_height_mean': np.mean(tree_height[region.coords[:, 0], region.coords[:, 1]]),
            'building_height_mean': np.mean(building_height[region.coords[:, 0], region.coords[:, 1]]),
            'dem_mean': np.mean(dem[region.coords[:, 0], region.coords[:, 1]]),
            'timestamp': row['timestamp']
        }


        for col in csv_data.columns[1:]:
            features[col] = row[col]

        features_list.append(features)

all_features_df = pd.DataFrame(features_list)

# 将所有特征数据保存到一个CSV文件中
all_features_df.to_csv('/content/drive/MyDrive/UTCI_prediction/data/feature_output.csv', index=False)

提取UTCI数据，前期都是数据预处理部分，期待把两个数据集合一

In [ ]:
import os
import pandas as pd
import numpy as np
import rasterio
from skimage.measure import label, regionprops
from datetime import datetime, timedelta

def read_utci_from_tif(file_path):
    """专门用于从TIF文件读取UTCi数据的函数。"""
    with rasterio.open(file_path) as src:
        utci_data = src.read(1)
    return utci_data

def segment_by_landcover(image):
    """根据土地利用的唯一分类值进行分割。"""
    labels = label(image)
    return labels

# 读取土地利用数据并进行分割
landcover_path = '/content/drive/MyDrive/UTCI_prediction/data/spatial_images/landcover9.tif'
landcover = read_utci_from_tif(landcover_path)
segmented_landcover = segment_by_landcover(landcover)

# 新数据集文件夹路径
new_dataset_folder = '/content/drive/MyDrive/UTCI_prediction/data/output_images'
features_list = []


initial_timestamp = datetime.strptime("2022-07-03-00", "%Y-%m-%d-%H")
time_increment = timedelta(hours=1)

# 遍历新数据集中的每个文件
for index, filename in enumerate(sorted(os.listdir(new_dataset_folder))):
    if filename.endswith('.tif'):
        new_data_path = os.path.join(new_dataset_folder, filename)
        new_data = read_utci_from_tif(new_data_path)


        current_timestamp = initial_timestamp + index * time_increment
        timestamp_str = current_timestamp.strftime("%Y-%m-%d-%H")

        regions = regionprops(segmented_landcover, intensity_image=new_data)
        for region in regions:
            utc_values = new_data[region.coords[:, 0], region.coords[:, 1]]
            utc_mean = np.mean(utc_values) if utc_values.size > 0 else np.nan

            features = {
                'region_id': region.label,
                'UTCI': utc_mean,
                'timestamp': timestamp_str
            }

            features_list.append(features)

all_features_df = pd.DataFrame(features_list)
output_csv_path = '/content/drive/MyDrive/UTCI_prediction/data/UTCI_feature.csv'
all_features_df.to_csv(output_csv_path, index=False)

这部分开始训练模型，前面的数据准备工作已经做好了

transformer模型

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Concatenate, Input, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Embedding
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f"Training on GPU: {physical_devices[0].name}")
else:
    print("No GPU found, using CPU for training")

# 加载数据
input_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/feature_output.csv')
utci_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/utci_feature.csv')

# 合并数据
merged_data = pd.merge(input_data, utci_data, on=['region_id','timestamp'])
merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'], format='%Y-%m-%d-%H')
data_size = merged_data.shape
print(f"DataFrame 行数: {data_size[0]}, 列数: {data_size[1]}")

# 数据标准化（包括GHI, DNI, DHI）
scaler = StandardScaler()
numeric_features = merged_data.drop(columns=['UTCI', 'timestamp', 'region_id'])
X_scaled = scaler.fit_transform(numeric_features)
y = merged_data['UTCI'].values

# 确保 region_id 和 y 的数据类型
region_ids = merged_data['region_id'].astype(np.int32).values
y = y.astype(np.float32)

# 按8:1:1划分数据集
X_train_numeric, X_temp_numeric, X_train_region, X_temp_region, y_train, y_temp = train_test_split(
    X_scaled, region_ids, y, test_size=0.2, random_state=42)

X_val_numeric, X_test_numeric, X_val_region, X_test_region, y_val, y_test = train_test_split(
    X_temp_numeric, X_temp_region, y_temp, test_size=0.5, random_state=42)

# 确定唯一的 region_id 数量
num_regions = merged_data['region_id'].nunique()
embedding_dim = 64  # 嵌入向量的维度

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, mlp_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.mlp = tf.keras.Sequential([
            Dense(mlp_dim, activation='relu'),
            Dense(d_model),
        ])
        self.dropout1 = Dropout(dropout)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.dropout2 = Dropout(dropout)
        self.norm2 = LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(inputs + attn_output)
        mlp_output = self.mlp(out1)
        mlp_output = self.dropout2(mlp_output, training=training)
        return self.norm2(out1 + mlp_output)

def build_model(input_shape, num_regions, embedding_dim):
    inputs_numeric = Input(shape=(input_shape[1],), name='numeric_inputs')
    inputs_region = Input(shape=(), name='region_input', dtype=tf.int32)

    region_embedding = Embedding(input_dim=num_regions + 1, output_dim=embedding_dim)(inputs_region)
    region_embedding = Flatten()(region_embedding)

    combined_inputs = Concatenate()([inputs_numeric, region_embedding])

    combined_inputs = Lambda(lambda x: tf.expand_dims(x, axis=1))(combined_inputs)

    transformer_output = TransformerBlock(d_model=combined_inputs.shape[-1], num_heads=4, mlp_dim=128)(combined_inputs)
    flattened_output = Flatten()(transformer_output)
    dense_output = Dense(64, activation='relu')(flattened_output)
    output = Dense(1)(dense_output)

    model = Model(inputs=[inputs_numeric, inputs_region], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

input_shape = X_train_numeric.shape

model = build_model(input_shape=input_shape, num_regions=num_regions, embedding_dim=embedding_dim)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

class PredictionHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.epoch_train_pred = []
        self.epoch_val_pred = []

    def on_epoch_end(self, epoch, logs=None):
        train_pred = self.model.predict({'numeric_inputs': X_train_numeric, 'region_input': X_train_region}, verbose=0)
        val_pred = self.model.predict({'numeric_inputs': X_val_numeric, 'region_input': X_val_region}, verbose=0)
        self.epoch_train_pred.append(train_pred)
        self.epoch_val_pred.append(val_pred)

prediction_history = PredictionHistory()

history = model.fit(
    {'numeric_inputs': X_train_numeric, 'region_input': X_train_region},
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=({'numeric_inputs': X_val_numeric, 'region_input': X_val_region}, y_val),
    callbacks=[early_stopping, prediction_history]
)

# 保存模型权重
model.save_weights('/content/drive/MyDrive/Transformer_weights.weights.h5')

# 评估模型性能
y_pred = model.predict({'numeric_inputs': X_test_numeric, 'region_input': X_test_region})
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 保存评估数据到 CSV
history_df = pd.DataFrame(history.history)
history_df['mse'] = mse
history_df['r2'] = r2
history_df.to_csv('/content/drive/MyDrive/transformer_evaluation_data.csv', index=False)

# 打印和绘制结果
print(f'MSE: {mse}, R^2: {r2}')

# 绘制训练和验证损失曲线
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 绘制训练和验证MAE曲线
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

# 绘制预测值与真实值随epoch的变化情况
def plot_predictions(epoch_predictions, y_true, title):
    plt.figure(figsize=(10, 5))
    for epoch, preds in enumerate(epoch_predictions):
        plt.plot(y_true, preds, 'o', label=f'Epoch {epoch+1}')
    plt.plot(y_true, y_true, 'r--', label='True values')
    plt.title(title)
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.legend()
    plt.show()

plot_predictions(prediction_history.epoch_train_pred, y_train, 'Training Predictions vs True Values')
plot_predictions(prediction_history.epoch_val_pred, y_val, 'Validation Predictions vs True Values')

结果对比

In [ ]:
# 绘制验证集预测结果与真实结果的对比
def plot_comparison(y_true, y_pred, title='Validation Set: Predictions vs True Values'):
    plt.figure(figsize=(10, 6))
    plt.plot(y_true, label='True Values', marker='o', linestyle='-', color='blue')
    plt.plot(y_pred, label='Predicted Values', marker='x', linestyle='--', color='red')
    plt.title(title)
    plt.xlabel('Sample Index')
    plt.ylabel('UTCI Value')
    plt.legend()
    plt.grid(True)
    plt.show()

# 获取验证集的预测结果
y_val_pred = model.predict({'numeric_inputs': X_val_numeric, 'region_input': X_val_region})

# 绘制对比图
plot_comparison(y_val, y_val_pred.flatten(), title='Validation Set: Predictions vs True Values')

LSTM算法

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Concatenate, Input, LSTM, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# 加载数据
input_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/feature_output.csv')
utci_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/utci_feature.csv')

# 合并数据
merged_data = pd.merge(input_data, utci_data, on=['region_id', 'timestamp'])
merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'], format='%Y-%m-%d-%H')

# 数据标准化
scaler = StandardScaler()
numeric_features = merged_data.drop(columns=['UTCI', 'timestamp', 'region_id'])
X_scaled = scaler.fit_transform(numeric_features)
y = merged_data['UTCI'].values

# 确保 region_id 和 y 的数据类型
region_ids = merged_data['region_id'].astype(np.int32).values  # 确保为 int32 类型
y = y.astype(np.float32)  # 确保 y 为 float32 类型

# 按8:1:1划分数据集
X_train_numeric, X_temp_numeric, X_train_region, X_temp_region, y_train, y_temp = train_test_split(
    X_scaled, region_ids, y, test_size=0.2, random_state=42)

X_val_numeric, X_test_numeric, X_val_region, X_test_region, y_val, y_test = train_test_split(
    X_temp_numeric, X_temp_region, y_temp, test_size=0.5, random_state=42)

# 确定唯一的 region_id 数量
num_regions = merged_data['region_id'].nunique()
embedding_dim = 64  # 嵌入向量的维度

# 构建模型
def build_model(input_shape, num_regions, embedding_dim):
    inputs_numeric = Input(shape=(input_shape[1],), name='numeric_inputs')
    inputs_region = Input(shape=(), name='region_input', dtype=tf.int32)

    region_embedding = Embedding(input_dim=num_regions + 1, output_dim=embedding_dim)(inputs_region)
    region_embedding = Flatten()(region_embedding)

    combined_inputs = Concatenate()([inputs_numeric, region_embedding])

    # 使用 Lambda 层包装 tf.expand_dims
    combined_inputs = Lambda(lambda x: tf.expand_dims(x, axis=1))(combined_inputs)

    lstm_output = LSTM(64)(combined_inputs)  # 替换为LSTM层
    dense_output = Dense(64, activation='relu')(lstm_output)
    output = Dense(1)(dense_output)

    model = Model(inputs=[inputs_numeric, inputs_region], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# 获取输入形状
input_shape = X_train_numeric.shape

# 构建和训练模型
model = build_model(input_shape=input_shape, num_regions=num_regions, embedding_dim=embedding_dim)

# 设置 EarlyStopping 回调
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 记录每个 epoch 的预测值和真实值
class PredictionHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.epoch_train_pred = []
        self.epoch_val_pred = []

    def on_epoch_end(self, epoch, logs=None):
        train_pred = self.model.predict({'numeric_inputs': X_train_numeric, 'region_input': X_train_region}, verbose=0)
        val_pred = self.model.predict({'numeric_inputs': X_val_numeric, 'region_input': X_val_region}, verbose=0)
        self.epoch_train_pred.append(train_pred)
        self.epoch_val_pred.append(val_pred)

prediction_history = PredictionHistory()

history = model.fit(
    {'numeric_inputs': X_train_numeric, 'region_input': X_train_region},
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=({'numeric_inputs': X_val_numeric, 'region_input': X_val_region}, y_val),
    callbacks=[early_stopping, prediction_history]
)

# 保存模型权重
model.save_weights('/content/drive/MyDrive/LSTM_weights.weights.h5')

# 评估模型性能
y_pred = model.predict({'numeric_inputs': X_test_numeric, 'region_input': X_test_region})
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 保存评估数据到 CSV
history_df = pd.DataFrame(history.history)
history_df['mse'] = mse
history_df['r2'] = r2
history_df.to_csv('/content/drive/MyDrive/LSTM_evaluation_data_LSTM.csv', index=False)

# 打印和绘制结果
print(f'MSE: {mse}, R^2: {r2}')

# 绘制训练和验证损失曲线
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 绘制训练和验证MAE曲线
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

# 绘制验证集预测结果与真实结果的对比
def plot_comparison(y_true, y_pred, title='Validation Set: Predictions vs True Values'):
    plt.figure(figsize=(10, 6))
    plt.plot(y_true, label='True Values', marker='o', linestyle='-', color='blue')
    plt.plot(y_pred, label='Predicted Values', marker='x', linestyle='--', color='red')
    plt.title(title)
    plt.xlabel('Sample Index')
    plt.ylabel('UTCI Value')
    plt.legend()
    plt.grid(True)
    plt.show()

# 获取验证集的预测结果
y_val_pred = model.predict({'numeric_inputs': X_val_numeric, 'region_input': X_val_region})

# 绘制对比图
plot_comparison(y_val, y_val_pred.flatten(), title='Validation Set: Predictions vs True Values')

1D卷积神经网络（1D CNN）

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Concatenate, Input, Conv1D, GlobalMaxPooling1D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# 加载数据
input_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/feature_output.csv')
utci_data = pd.read_csv('/content/drive/MyDrive/UTCI_prediction/data/utci_feature.csv')

# 合并数据
merged_data = pd.merge(input_data, utci_data, on=['region_id', 'timestamp'])
merged_data['timestamp'] = pd.to_datetime(merged_data['timestamp'], format='%Y-%m-%d-%H')

# 数据标准化
scaler = StandardScaler()
numeric_features = merged_data.drop(columns=['UTCI', 'timestamp', 'region_id'])
X_scaled = scaler.fit_transform(numeric_features)
y = merged_data['UTCI'].values

# 确保 region_id 和 y 的数据类型
region_ids = merged_data['region_id'].astype(np.int32).values
y = y.astype(np.float32)

# 按8:1:1划分数据集
X_train_numeric, X_temp_numeric, X_train_region, X_temp_region, y_train, y_temp = train_test_split(
    X_scaled, region_ids, y, test_size=0.2, random_state=42)

X_val_numeric, X_test_numeric, X_val_region, X_test_region, y_val, y_test = train_test_split(
    X_temp_numeric, X_temp_region, y_temp, test_size=0.5, random_state=42)

# 确定唯一的 region_id 数量
num_regions = merged_data['region_id'].nunique()
embedding_dim = 64  # 嵌入向量的维度

# 构建1D CNN模型
def build_model(input_shape, num_regions, embedding_dim):
    inputs_numeric = Input(shape=(input_shape[1],), name='numeric_inputs')
    inputs_region = Input(shape=(), name='region_input', dtype=tf.int32)

    region_embedding = Embedding(input_dim=num_regions + 1, output_dim=embedding_dim)(inputs_region)
    region_embedding = Flatten()(region_embedding)

    combined_inputs = Concatenate()([inputs_numeric, region_embedding])

    # 使用 Lambda 层包装 tf.expand_dims
    combined_inputs = Lambda(lambda x: tf.expand_dims(x, axis=2))(combined_inputs)

    conv_output = Conv1D(filters=64, kernel_size=3, activation='relu')(combined_inputs)
    pooled_output = GlobalMaxPooling1D()(conv_output)
    dense_output = Dense(64, activation='relu')(pooled_output)
    output = Dense(1)(dense_output)

    model = Model(inputs=[inputs_numeric, inputs_region], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# 获取输入形状
input_shape = X_train_numeric.shape

# 构建和训练模型
model = build_model(input_shape=input_shape, num_regions=num_regions, embedding_dim=embedding_dim)

# 设置 EarlyStopping 回调
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    {'numeric_inputs': X_train_numeric, 'region_input': X_train_region},
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=({'numeric_inputs': X_val_numeric, 'region_input': X_val_region}, y_val),
    callbacks=[early_stopping]
)

# 评估模型性能
y_pred = model.predict({'numeric_inputs': X_test_numeric, 'region_input': X_test_region})
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 保存模型权重
model.save_weights('/content/drive/MyDrive/CNN_weights.h5')

# 保存评估数据到 CSV
history_df = pd.DataFrame(history.history)
history_df['mse'] = mse
history_df['r2'] = r2
history_df.to_csv('/content/drive/MyDrive/CNN_evaluation_data.csv', index=False)

# 打印和绘制结果
print(f'MSE: {mse}, R^2: {r2}')

# 绘制训练和验证损失曲线
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

 GRU (Gated Recurrent Unit)

In [ ]:
from tensorflow.keras.layers import GRU

# 构建GRU模型
def build_model(input_shape, num_regions, embedding_dim):
    inputs_numeric = Input(shape=(input_shape[1],), name='numeric_inputs')
    inputs_region = Input(shape=(), name='region_input', dtype=tf.int32)

    region_embedding = Embedding(input_dim=num_regions + 1, output_dim=embedding_dim)(inputs_region)
    region_embedding = Flatten()(region_embedding)

    combined_inputs = Concatenate()([inputs_numeric, region_embedding])

    # 使用 Lambda 层包装 tf.expand_dims
    combined_inputs = Lambda(lambda x: tf.expand_dims(x, axis=1))(combined_inputs)

    gru_output = GRU(64)(combined_inputs)
    dense_output = Dense(64, activation='relu')(gru_output)
    output = Dense(1)(dense_output)

    model = Model(inputs=[inputs_numeric, inputs_region], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# 获取输入形状
input_shape = X_train_numeric.shape

# 构建和训练模型
model = build_model(input_shape=input_shape, num_regions=num_regions, embedding_dim=embedding_dim)

# 设置 EarlyStopping 回调
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    {'numeric_inputs': X_train_numeric, 'region_input': X_train_region},
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=({'numeric_inputs': X_val_numeric, 'region_input': X_val_region}, y_val),
    callbacks=[early_stopping]
)

# 后续步骤与之前的示例相同

混合模型：CNN + RNN

In [ ]:
from tensorflow.keras.layers import GRU, Conv1D, GlobalMaxPooling1D

# 构建CNN + GRU混合模型
def build_model(input_shape, num_regions, embedding_dim):
    inputs_numeric = Input(shape=(input_shape[1],), name='numeric_inputs')
    inputs_region = Input(shape=(), name='region_input', dtype=tf.int32)

    region_embedding = Embedding(input_dim=num_regions + 1, output_dim=embedding_dim)(inputs_region)
    region_embedding = Flatten()(region_embedding)

    combined_inputs = Concatenate()([inputs_numeric, region_embedding])

    combined_inputs = Lambda(lambda x: tf.expand_dims(x, axis=2))(combined_inputs)

    conv_output = Conv1D(filters=64, kernel_size=3, activation='relu')(combined_inputs)
    pooled_output = GlobalMaxPooling1D()(conv_output)

    lstm_output = GRU(64)(Lambda(lambda x: tf.expand_dims(x, axis=1))(pooled_output))
    dense_output = Dense(64, activation='relu')(lstm_output)
    output = Dense(1)(dense_output)

    model = Model(inputs=[inputs_numeric, inputs_region], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# 获取输入形状
input_shape = X_train_numeric.shape

# 构建和训练模型
model = build_model(input_shape=input_shape, num_regions=num_regions, embedding_dim=embedding_dim)

# 设置 EarlyStopping 回调
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    {'numeric_inputs': X_train_numeric, 'region_input': X_train_region},
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=({'numeric_inputs': X_val_numeric, 'region_input': X_val_region}, y_val),
    callbacks=[early_stopping]
)

# 后续步骤与之前的示例相同

MLP (Multi-Layer Perceptron)

In [ ]:
# 构建MLP模型
def build_model(input_shape, num_regions, embedding_dim):
    inputs_numeric = Input(shape=(input_shape[1],), name='numeric_inputs')
    inputs_region = Input(shape=(), name='region_input', dtype=tf.int32)

    region_embedding = Embedding(input_dim=num_regions + 1, output_dim=embedding_dim)(inputs_region)
    region_embedding = Flatten()(region_embedding)

    combined_inputs = Concatenate()([inputs_numeric, region_embedding])

    dense_output = Dense(128, activation='relu')(combined_inputs)
    dense_output = Dropout(0.2)(dense_output)
    dense_output = Dense(64, activation='relu')(dense_output)
    output = Dense(1)(dense_output)

    model = Model(inputs=[inputs_numeric, inputs_region], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# 获取输入形状
input_shape = X_train_numeric.shape

# 构建和训练模型
model = build_model(input_shape=input_shape, num_regions=num_regions, embedding_dim=embedding_dim)

# 设置 EarlyStopping 回调
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    {'numeric_inputs': X_train_numeric, 'region_input': X_train_region},
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=({'numeric_inputs': X_val_numeric, 'region_input': X_val_region}, y_val),
    callbacks=[early_stopping]
)

# 后续步骤与之前的示例相同

以下是对比1D CNN、GRU、CNN+GRU、MLP几种模型在时序数据处理中的优缺点和适用场景的表格：

| **模型**        | **架构类型** | **优点** | **缺点** | **适用场景** |
|-----------------|--------------|----------|----------|--------------|
| **1D CNN**      | 卷积神经网络 (CNN) | - 擅长提取局部特征<br>- 计算效率高，适合并行处理<br>- 能捕获短期依赖关系 | - 不擅长处理长时间依赖<br>- 可能无法充分利用时序信息 | - 时序信号处理<br>- 短时间依赖特征提取 |
| **GRU**         | 循环神经网络 (RNN) | - 较轻量的RNN结构<br>- 擅长捕获长时间依赖<br>- 比LSTM更快，参数更少 | - 计算复杂度较高，难以并行<br>- 在非常长序列上仍有局限性 | - 长时间依赖任务<br>- 时间序列预测<br>- 自然语言处理 |
| **CNN + GRU**   | 混合模型 (CNN + RNN) | - 结合了CNN和RNN的优点<br>- CNN处理局部特征，RNN处理全局时间依赖<br>- 通常表现优于单一模型 | - 复杂度较高<br>- 需要更多计算资源<br>- 训练时间长 | - 需要同时提取局部和全局特征的任务<br>- 复杂的时序数据处理 |
| **MLP**         | 多层感知器 (全连接神经网络) | - 结构简单，易于实现<br>- 适合小型数据集<br>- 训练速度快 | - 不适合捕捉时间依赖关系<br>- 无法充分利用时序数据的特性 | - 非时间依赖的任务<br>- 简单的回归或分类任务 |

### 详细说明：
1. **1D CNN**:
   - **优点**：1D CNN 通过卷积操作能够有效地提取序列中的局部模式，特别适用于具有局部依赖关系的时序数据。此外，由于卷积运算的高度并行性，计算效率较高，适合在较大数据集上进行快速训练。
   - **缺点**：1D CNN 主要关注局部特征，而不能很好地捕捉长时间的依赖关系。如果你的数据具有较长的时间依赖性，CNN 可能会丢失一些全局信息。
   - **适用场景**：适合处理短期依赖的时序信号，如语音处理或传感器数据分析。

2. **GRU**:
   - **优点**：GRU 是 RNN 的一种变体，能够有效地捕捉长时间依赖关系，但相比 LSTM 参数更少，计算速度更快。GRU 在处理长序列时表现良好，并且由于结构较简单，适合快速原型开发。
   - **缺点**：尽管 GRU 可以处理长时间依赖，但与 CNN 相比，计算复杂度较高，并且由于其序列化处理的特性，无法很好地并行化，训练时间可能较长。
   - **适用场景**：适用于时间依赖性强的任务，如时间序列预测、文本生成等。

3. **CNN + GRU**:
   - **优点**：这种混合模型结合了 CNN 和 GRU 的优点，CNN 负责提取序列的局部特征，GRU 处理全局的时间依赖关系。通常情况下，这种组合模型在复杂的时序数据处理任务中表现更优。
   - **缺点**：由于组合了两种模型，这种方法的复杂度较高，需要更多的计算资源和更长的训练时间。
   - **适用场景**：适用于需要同时处理局部特征和全局依赖的复杂时序任务，如视频分析或多模态数据分析。

4. **MLP**:
   - **优点**：MLP 是一种结构简单的全连接神经网络，适合处理小型数据集，训练速度快，适合快速测试模型。
   - **缺点**：由于 MLP 无法处理序列数据的时间依赖性，因此在时序数据任务中表现较差，无法充分利用时间序列中的相关信息。
   - **适用场景**：适用于非时序性任务，或简单的回归和分类问题，如预测单一变量或分类静态数据。

### 总结：
- **如果数据具有显著的时间依赖性**，如时间序列预测任务，**GRU** 和 **CNN+GRU** 是较好的选择。GRU 适合长时间依赖性的数据，而 CNN+GRU 则能同时捕获局部和全局特征。
- **如果你只关心短时间依赖关系或数据的局部模式**，**1D CNN** 是一个高效的选择。
- **如果数据是静态的或时间依赖性不强**，**MLP** 可以提供一个简单有效的解决方案。

根据你的具体应用场景，可以选择其中一种或结合多种方法进行实验，找到最适合的模型。